# softmax回归从零开始实现

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from IPython import display
from matplotlib import pyplot as plt
import random

## 1 获取和读取数据

In [2]:
#构建多分类Fashion-minst数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True,
download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False,
download=True, transform=transforms.ToTensor())
batch_size = 256
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True,
num_workers=0)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False,
num_workers=0)

D:\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 2 初始化模型参数

In [3]:
#初始化参数w，b
# 输入与输出
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0,0.01,(num_inputs,num_outputs)),dtype=torch.float)
b = torch.zeros(num_outputs,dtype = torch.float)
#开启梯度track
W.requires_grad_(requires_grad = True)
b.requires_grad_(requires_grad = True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

## 3 实现softmax运算

In [4]:
#定义softmax函数
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim = 1, keepdim=True)
    return X_exp / partition


## 4 定义模型

In [5]:
#手动实现定义模型
def net(X):
    #torch.mm  矩阵相乘  view（）改变矩阵维度为1行 num_input列
    f_x = torch.mm(X.view((-1,num_inputs)),W) + b
    return softmax(f_x)

## 5 定义损失函数

In [6]:
#手动实现交叉熵损失函数
def cross_entropy(y_hat, y):
    #1按行取
    return -torch.log(y_hat.gather(1, y.view(-1,1)))


## 6 定义优化算法

In [7]:
#优化函数
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data


## 7 计算分类准确率

In [8]:
def evaluate_accuracy(data_iter,net):
    acc_sum,n = 0.0,0
    for X,y in data_iter:
        #print(len(X)) 小批量数据集 每个X中有 256个图像
        #print((net(X).argmax(dim=1)==y).float().sum().item())
       acc_sum += (net(X).argmax(dim=1)==y).float().sum().item()
       n+=y.shape[0]
    return acc_sum/n

## 8 训练模型

In [9]:
def train(net, train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,optimizer = None):
    for epoch in range(num_epochs):
        #模型训练次数 num_epochs次
        train_l_num, train_acc_num,n = 0.0,0.0,0
        for X,y in train_iter:
            #X 为小批量256个图像 1*28*28 y为标签
            # 计算X softmax下的值   与损失函数值
            y_hat = net(X)
            l = loss(y_hat,y).sum()
            l.backward()
            sgd([W, b], lr, batch_size) # 使用小批量随机梯度下降迭代模型参数
            W.grad.data.zero_() # 梯度清零
            b.grad.data.zero_()
            #计算每个epoch的loss
            train_l_num += l.item()
            #计算训练样本的准确率
            train_acc_num += (y_hat.argmax(dim=1)==y).sum().item()
            #每一个epoch的所有样本数
            n+= y.shape[0]
        #计算测试样本的准确率
        test_acc = evaluate_accuracy(test_iter,net)
        print('epoch %d, loss %.4f,train_acc %.3f,test_acc %.3f'%(epoch+1,train_l_num/n, train_acc_num/n, test_acc))


## 9 预测

In [10]:
num_epochs ,lr = 30,0.1
train(net, train_iter, test_iter, cross_entropy, num_epochs,batch_size, [W, b], lr)


epoch 1, loss 0.7818,train_acc 0.752,test_acc 0.794
epoch 2, loss 0.5694,train_acc 0.813,test_acc 0.812
epoch 3, loss 0.5249,train_acc 0.826,test_acc 0.819
epoch 4, loss 0.5014,train_acc 0.833,test_acc 0.824
epoch 5, loss 0.4849,train_acc 0.837,test_acc 0.826
epoch 6, loss 0.4744,train_acc 0.841,test_acc 0.830
epoch 7, loss 0.4653,train_acc 0.843,test_acc 0.827
epoch 8, loss 0.4578,train_acc 0.845,test_acc 0.828
epoch 9, loss 0.4525,train_acc 0.847,test_acc 0.833
epoch 10, loss 0.4476,train_acc 0.848,test_acc 0.835
epoch 11, loss 0.4430,train_acc 0.850,test_acc 0.837
epoch 12, loss 0.4389,train_acc 0.850,test_acc 0.835
epoch 13, loss 0.4365,train_acc 0.852,test_acc 0.834
epoch 14, loss 0.4331,train_acc 0.853,test_acc 0.836
epoch 15, loss 0.4306,train_acc 0.853,test_acc 0.839
epoch 16, loss 0.4281,train_acc 0.854,test_acc 0.838
epoch 17, loss 0.4265,train_acc 0.854,test_acc 0.838
epoch 18, loss 0.4237,train_acc 0.855,test_acc 0.839
epoch 19, loss 0.4220,train_acc 0.856,test_acc 0.839
ep